In [636]:
import pgeocode
import folium
import pandas as pd
import requests
from functools import reduce

In [933]:
username=''

In [829]:
class Weather:
    '''
    This class gets weather forecast with following methods-
    1. weather_daily -> daily weather attributes for defined days forecast
    2. weather_hourly -> hourly weather attributes for defined days forecast
    3. weather_analyse -> collects weather data once both methods are ran(dependent on daily and hourly methods)
    4. weather_condition -> spits a tuple of 1s and 0s with condtions derived from above methods with comparison to thresholds a user inputs.
    '''
    def __init__(self, latitude=None, longitude=None):
        '''
        Constructor method. Takes following inputs-
        latitude, longitude
        Also uses caching for wdaily(weather_daily), whourly(weather_hourly) and wdata(complete weather_data)
        default days are set to 7 for weather data collection
        API used- open-meto
        '''
        self._lat = latitude
        self._lon = longitude
        self.wdaily_cache = {}
        self.whourly_cache = {}
        self.wkey = (self._lat, self._lon)
        self.rain_sum = 0
        self.rain_hrs = 0
        self.days = 7
        self.wdata = {}
        
    def weather_daily(self, *attr, days=None):
        '''
        days: how many days of forecast? max=16 min=1
        attr: daily attributes like precipitation_sum, precipitation_hours(both defaults)
        returns: a dict of daily weather attributes which is populated in the cache(wdaily) for re-use
        
        '''
        if not days:
            days = self.days
        if not attr:
            attr = ('precipitation_sum', 'precipitation_hours', 'temperature_2m_max')
        attributes = ','.join(attr)
        #   Check cache
        if self.wkey in self.wdaily_cache:
            print('WCache Hit')
            return self.wdaily_cache[self.wkey]
        wurl = f'https://api.open-meteo.com/v1/forecast?latitude={self._lat}&longitude={self._lon}&daily={attributes}&forecast_days={days}&timezone=Europe%2FLondon'
        #         print(wurl)
        daily_data = requests.get(wurl)
        #   Populating the cache
        self.wdaily_cache[self.wkey] = daily_data.json()
        return self.wdaily_cache[self.wkey]
    
    def weather_hourly(self, *attr, days=None):
        '''
        days: how many days of forecast? max=16 min=1
        attr: hourly attributes like cloudcover, precipitation(both defaults)
        returns: a dict of hourly weather attributes which is populated in the cache(whourly) for re-use
        '''
        if not days:
            days = self.days
        if not attr:
            attr = ('cloudcover', 'precipitation')
        attributes = ','.join(attr)        
        #   Check cache
        if self.wkey in self.whourly_cache:
            print('WCache Hit')
            return self.whourly_cache[self.wkey]
        wurl = f'https://api.open-meteo.com/v1/forecast?latitude={self._lat}&longitude={self._lon}&hourly={attributes}&forecast_days={days}&timezone=Europe%2FLondon'
        #         print(wurl)
        hourly_data = requests.get(wurl)
        #   Populating the cache
        self.whourly_cache[self.wkey] = hourly_data.json()
        return self.whourly_cache[self.wkey]    
    def weather_analyse(self):
        '''
        Collects Weather data once daily and hourly methods are run. Please run both methods to collect combined data.
        returns: a dict of weather_data(wdata)
        '''
        if (self.wdaily_cache and self.whourly_cache):
            precipitation_sum = self.wdaily_cache[self.wkey]['daily']['precipitation_sum']
            precipitation_hours = self.wdaily_cache[self.wkey]['daily']['precipitation_hours']
            temp_max = self.wdaily_cache[self.wkey]['daily']['temperature_2m_max']
            cloud_cover = self.whourly_cache[self.wkey]['hourly']['cloudcover']
            self.precipitation_sum = sum(precipitation_sum)/len(precipitation_sum)
            self.precipitation_hours = sum(precipitation_hours)/len(precipitation_hours)
            self.temp_max = reduce(lambda x,y: x+y, temp_max)/len(temp_max)
            self.cloud_cover = sum(cloud_cover)/len(cloud_cover)
            self.wdata = {'precipitation_sum': self.precipitation_sum, 'precipitation_hours':self.precipitation_hours, 'temp_max': self.temp_max, 'cloud_cover': self.cloud_cover}
            return self.wdata
        elif self.wdaily_cache:
            precipitation_sum = self.wdaily_cache[self.wkey]['daily']['precipitation_sum']
            precipitation_hours = self.wdaily_cache[self.wkey]['daily']['precipitation_hours']
            temp_max = self.wdaily_cache[self.wkey]['daily']['temperature_2m_max']
            self.wdata = {'precipitation_sum': self.precipitation_sum, 'precipitation_hours':self.precipitation_hours, 'temp_max': self.temp_max}
            return self.wdata
        else:
            cloud_cover = self.whourly_cache[self.wkey]['hourly']['cloudcover']
            self.cloud_cover = sum(cloud_cover)/len(cloud_cover)
            self.wdata = {'temp_max': self.temp_max}
            return self.wdata
        
    def weather_condition(self, precipitation_sum, precipitation_hours, temp_max, cloud_cover):
        '''
        Inputs: Thresholds for precipitation_sum, precipitation_hours, temp_max, cloud_cover
        Returns: A Tuple which compares the thresholds with actual weather data and returns either 1 or 0
        Depends on weather_daily and weather_hourly
        '''
        return (int(self.precipitation_sum<=precipitation_sum), int(self.precipitation_hours<=precipitation_hours), int(self.temp_max >= temp_max), int(self.cloud_cover <= cloud_cover))

    
        
        

In [998]:
class Findme():
    
    def __init__(self, country_code, city, username):
        '''
        Constructor method for Findme Class. Takes following inputs-
        country_code, city(city_name) and user_name. Please register yourself on geonames for this!
        Also uses caching for collecting city data
        API used- Geonames free tier and pgecode library
        '''
        self._country_code = country_code
        self._city = city
        self._username = username
        self._lcodes = pgeocode.Nominatim(self._country_code).query_location(self._city).head(1)[['latitude', 'longitude']]
        self._lat = self._lcodes['latitude'].values[0]
        self._lon = self._lcodes['longitude'].values[0]
        self.cache = {}
        self.wdaily_cache = {}
        self.wfetch = {}
    def nearby_place(self, radius, city_type, max_rows):
        '''
        params: radius(max is 300), city_type(basis on population, for accepted values check geonames), max_rows(max is 500)
        returns -> returns a list of all cities in the radius defined.
        Re-initilise objects to renew the cache in case radius other attributes need to change
        'cache' is maintained for storing results of all nearby cities
        '''
        if self._city in self.cache:
            print('Cache Hit')
            return self.cache[self._city]
        url = f'http://api.geonames.org/findNearbyPlaceNameJSON?lat={self._lat}&lng={self._lon}&radius={radius}&maxRows={max_rows}&cities={city_type}&username={self._username}'
        response = requests.get(url)
        data = response.json()
#         Populate the cache
        self.cache[self._city] = list(data.values())[0]
        print(f'Caching for {self._city}')
        return self.cache[self._city]
    
    def bulk_weather(self, days=10, t_precipitation_sum=1, t_precipitation_hours=2, t_temp_max=24, t_cloud_cover=60):
        '''
        params: days(10 def), t_precipitation_sum=1, t_precipitation_hours=2, t_temp_max=24, t_cloud_cover=60. All default threshold values.
        returns -> a dict(where key is city_name) of weather_condition as tuple and weather_data as another dict
        'cache' is maintained for storing results of all cities checked.
        The thresholds can be changed and it'll retrieve data from the cache and revise it without performing any API calls
        '''        
        if len(self.wdaily_cache) == len(self.cache[self._city]):
            print('Wcache Hit')
            # checking all cities in the cache            
            for wcity in self.wdaily_cache:
            # retrieving the weather data from the dictionary -> it's populated as {'city_key': [weather_condition, weather_data]}                
                precipitation_sum = self.wdaily_cache[wcity][1]['precipitation_sum']
                precipitation_hours = self.wdaily_cache[wcity][1]['precipitation_hours']
                temp_max = self.wdaily_cache[wcity][1]['temp_max']
                cloud_cover = self.wdaily_cache[wcity][1]['cloud_cover']
            # comparing threshold values from the cache                
                weather_analysis = (int(precipitation_sum<=t_precipitation_sum), int(precipitation_hours<=t_precipitation_hours), int(temp_max >= t_temp_max), int(cloud_cover <= t_cloud_cover))
                self.wdaily_cache[wcity][0] = weather_analysis
            return self.wdaily_cache
        else:
            for wcity in self.cache[self._city]:
                weather = Weather(wcity['lat'], wcity['lng'])
                weather_data_daily = weather.weather_daily('precipitation_sum', 'precipitation_hours', 'temperature_2m_max', days=days)
                weather_data_hourly = weather.weather_hourly('cloudcover', days=days)
                weather_data_collect = weather.weather_analyse() #use a var to store raw data and use it in if statement
                weather_analysis = weather.weather_condition(t_precipitation_sum, t_precipitation_hours, t_temp_max, t_cloud_cover)
    #             Creating a city_key cuz there can be same city names nearby
                city_key = wcity['name']+'_'+str(wcity['geonameId'])
                self.wdaily_cache[city_key] = [weather_analysis, weather_data_collect, (wcity['lat'], wcity['lng'])]
            return self.wdaily_cache
        
    def bulk_fetch(self):
        sunny = [(1,1,1,1), (1,1,0,1), (0,1,1,1), (0,1,0,1)]
        cloudy = [(1,1,1,0), (1,1,0,0), (0,1,1,0)]
        self.wfetch = {}
        for city in self.wdaily_cache:
            if self.wdaily_cache[city][0] in sunny:
                self.wfetch[city] = [self.wdaily_cache[city][2], 'sunny']
            elif self.wdaily_cache[city][0] in cloudy:
                self.wfetch[city] = [self.wdaily_cache[city][2], 'cloudy']
        return self.wfetch
    
    def bulk_plot(self):
        city_lat = self._lat
        city_lon = self._lon
        map_weather = folium.Map(location=[city_lat, city_lon], zoom_start=10)
        for city_key in self.wfetch:
            lat = self.wfetch[city_key][0][0]
            lon = self.wfetch[city_key][0][1]
            city = city_key.split('_')[0]
            comment = self.wfetch[city_key][1]
            marker_popup = f"{city}<br>Weather: {comment}"
            folium.Marker([lat, lon], popup=marker_popup).add_to(map_weather)
        return map_weather

            
        

        

In [954]:
wedin = Weather(55.95, -3.20)

In [955]:
help(wedin.weather_daily)

Help on method weather_daily in module __main__:

weather_daily(*attr, days=None) method of __main__.Weather instance
    days: how many days of forecast? max=16 min=1
    attr: daily attributes like precipitation_sum, precipitation_hours(both defaults)



In [956]:
wedin.weather_daily('temperature_2m_max', 'precipitation_hours', 'precipitation_sum', days=10)

{'latitude': 55.96,
 'longitude': -3.2,
 'generationtime_ms': 0.3820657730102539,
 'utc_offset_seconds': 3600,
 'timezone': 'Europe/London',
 'timezone_abbreviation': 'BST',
 'elevation': 54.0,
 'daily_units': {'time': 'iso8601',
  'temperature_2m_max': '°C',
  'precipitation_hours': 'h',
  'precipitation_sum': 'mm'},
 'daily': {'time': ['2023-06-03',
   '2023-06-04',
   '2023-06-05',
   '2023-06-06',
   '2023-06-07',
   '2023-06-08',
   '2023-06-09',
   '2023-06-10',
   '2023-06-11',
   '2023-06-12'],
  'temperature_2m_max': [16.6,
   16.3,
   18.5,
   18.4,
   19.4,
   18.8,
   18.0,
   18.8,
   20.3,
   18.6],
  'precipitation_hours': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0],
  'precipitation_sum': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]}}

In [1009]:
hourly= wedin.weather_hourly('cloudcover', days=10)

WCache Hit


In [941]:
wedin.weather_analyse()

{'precipitation_sum': 0.1,
 'precipitation_hours': 0.4,
 'temp_max': 18.37,
 'cloud_cover': 57.47083333333333}

In [943]:
wedin.weather_condition(1, 1, 19, 50)

(1, 1, 0, 0)

In [590]:
help(Findme.nearby_place)

Help on function nearby_place in module __main__:

nearby_place(self, radius, city_type, max_rows)
    params: radius(max is 300), city_type(basis on population), max_rows(max is 500)
    returns -> returns a list of all cities in the radius defined.
    Re-initilise objects to renew the cache in case radius other attributes need to change



In [999]:
edinburgh = Findme('gb','Edinburgh',username)


In [945]:
london = Findme('gb','London',username)

In [1000]:
portree = Findme('gb', 'Portree', username)

In [984]:
edinburgh15k = Findme('gb','Edinburgh',username)


In [1010]:
ldata= london.nearby_place(100, 'cities15000', 500)

Cache Hit


In [1011]:
pdata= portree.nearby_place(100, 'cities1000', 500)

Cache Hit


In [1012]:
edata=edinburgh.nearby_place(200, 'cities15000', 500)

Cache Hit


In [1013]:
e15data= edinburgh15k.nearby_place(300, 'cities15000', 500)

Caching for Edinburgh


In [997]:
edinburgh.bulk_weather(t_precipitation_sum=0, t_precipitation_hours=0, t_temp_max=25, t_cloud_cover=30, days=14)

Wcache Hit


{'Edinburgh_2650225': [(0, 0, 0, 0),
  {'precipitation_sum': 0.4785714285714286,
   'precipitation_hours': 1.5714285714285714,
   'temp_max': 17.50714285714286,
   'cloud_cover': 58.67559523809524},
  ('55.95206', '-3.19648')],
 'Broxburn_2654480': [(0, 0, 0, 0),
  {'precipitation_sum': 0.3071428571428571,
   'precipitation_hours': 1.3571428571428572,
   'temp_max': 18.185714285714287,
   'cloud_cover': 59.93452380952381},
  ('55.93415', '-3.47133')],
 'Penicuik_2640465': [(0, 0, 0, 0),
  {'precipitation_sum': 0.4714285714285714,
   'precipitation_hours': 2.2142857142857144,
   'temp_max': 18.235714285714284,
   'cloud_cover': 60.07738095238095},
  ('55.83116', '-3.22608')],
 'Bonnyrigg_2655210': [(0, 0, 0, 0),
  {'precipitation_sum': 0.6142857142857142,
   'precipitation_hours': 1.7857142857142858,
   'temp_max': 17.11428571428571,
   'cloud_cover': 58.401785714285715},
  ('55.87329', '-3.1051')],
 'Livingston_2644204': [(0, 0, 0, 0),
  {'precipitation_sum': 0.3071428571428571,
   'pr

In [1006]:
portree.bulk_weather(t_precipitation_sum=0, t_precipitation_hours=0, t_temp_max=20, t_cloud_cover=40, days=14)

Wcache Hit


{'Portree_2640006': [(0, 0, 0, 0),
  {'precipitation_sum': 0.04285714285714286,
   'precipitation_hours': 0.42857142857142855,
   'temp_max': 16.985714285714288,
   'cloud_cover': 52.44345238095238},
  ('57.41288', '-6.19418')],
 'Benbecula_8299868': [(1, 1, 0, 0),
  {'precipitation_sum': 0.0,
   'precipitation_hours': 0.0,
   'temp_max': 14.421428571428569,
   'cloud_cover': 45.666666666666664},
  ('57.44737', '-7.34273')],
 'South Uist_8299850': [(1, 1, 0, 0),
  {'precipitation_sum': 0.0,
   'precipitation_hours': 0.0,
   'temp_max': 14.614285714285716,
   'cloud_cover': 42.291666666666664},
  ('57.24562', '-7.33337')],
 'North Uist_8299849': [(1, 1, 0, 0),
  {'precipitation_sum': 0.0,
   'precipitation_hours': 0.0,
   'temp_max': 13.171428571428573,
   'cloud_cover': 48.517857142857146},
  ('57.60581', '-7.34024')],
 'Ullapool_2635199': [(0, 0, 0, 0),
  {'precipitation_sum': 0.02142857142857143,
   'precipitation_hours': 0.21428571428571427,
   'temp_max': 16.742857142857144,
   'cl

In [925]:
london.bulk_weather(t_precipitation_sum=1, t_precipitation_hours=1, t_temp_max=24, t_cloud_cover=32, days=14)

Wcache Hit


{'Canary Wharf_6692280': [(1, 1, 0, 0),
  {'precipitation_sum': 0.32142857142857145,
   'precipitation_hours': 0.8571428571428571,
   'temp_max': 22.749999999999996,
   'cloud_cover': 36.788690476190474}],
 'Limehouse_2644497': [(1, 1, 0, 0),
  {'precipitation_sum': 0.32142857142857145,
   'precipitation_hours': 0.8571428571428571,
   'temp_max': 22.914285714285715,
   'cloud_cover': 36.791666666666664}],
 'Blackwall_2655438': [(1, 1, 0, 0),
  {'precipitation_sum': 0.32142857142857145,
   'precipitation_hours': 0.8571428571428571,
   'temp_max': 22.878571428571426,
   'cloud_cover': 36.788690476190474}],
 'Stepney_2636948': [(1, 1, 0, 0),
  {'precipitation_sum': 0.32142857142857145,
   'precipitation_hours': 0.8571428571428571,
   'temp_max': 22.914285714285715,
   'cloud_cover': 36.791666666666664}],
 'Bow_2655076': [(1, 1, 0, 0),
  {'precipitation_sum': 0.32142857142857145,
   'precipitation_hours': 0.8571428571428571,
   'temp_max': 22.914285714285715,
   'cloud_cover': 37.029761904

In [1007]:
portree.bulk_fetch()

{'Benbecula_8299868': [('57.44737', '-7.34273'), 'cloudy'],
 'South Uist_8299850': [('57.24562', '-7.33337'), 'cloudy'],
 'North Uist_8299849': [('57.60581', '-7.34024'), 'cloudy'],
 'Barra_8299866': [('56.98035', '-7.45731'), 'sunny']}

In [1008]:
portree.bulk_plot()

In [996]:
edinburgh.bulk_fetch()

{'Bathgate_2656169': [('55.90204', '-3.64398'), 'cloudy'],
 'Falkirk_2649723': [('56.0021', '-3.78535'), 'cloudy'],
 'Wishaw_2633765': [('55.76667', '-3.91667'), 'cloudy'],
 'Airdrie_2657613': [('55.86602', '-3.98025'), 'cloudy'],
 'Cumbernauld_2651715': [('55.94685', '-3.99051'), 'cloudy'],
 'Dalserf_2651558': [('55.73333', '-3.91667'), 'cloudy'],
 'Coatbridge_2652696': [('55.86216', '-4.02469'), 'cloudy'],
 'Motherwell_2642135': [('55.78924', '-3.99187'), 'cloudy'],
 'Bellshill_2655952': [('55.81667', '-4.01667'), 'cloudy'],
 'Viewpark_11592296': [('55.82737', '-4.0573'), 'cloudy'],
 'Hamilton_2647570': [('55.76667', '-4.03333'), 'cloudy'],
 'Blantyre_11592111': [('55.79634', '-4.09485'), 'cloudy'],
 'High Blantyre_2646979': [('55.78438', '-4.10007'), 'cloudy'],
 'Kirkintilloch_2645261': [('55.93933', '-4.15262'), 'cloudy'],
 'Irvine_2646032': [('55.6194', '-4.65508'), 'cloudy'],
 'Ayr_2656708': [('55.46273', '-4.63393'), 'cloudy'],
 'Bedlington_2656039': [('55.13061', '-1.59319'), '

In [988]:
edinburgh.bulk_plot()

In [602]:
bergen.wdaily_cache

{'Jellet Nord_11282303': (0.19999999999999998, 1.1428571428571428),
 'Eidemarken_11282305': (0.19999999999999998, 1.1428571428571428),
 'Bergenhus_9304173': (0.19999999999999998, 1.1428571428571428),
 'Vagsbunnen_11282302': (0.19999999999999998, 1.1428571428571428),
 'Skuteviken_9304415': (0.19999999999999998, 1.1428571428571428),
 'Bergen_3161732': (0.19999999999999998, 1.1428571428571428),
 'Sandviken_3140120': (0.19999999999999998, 1.1428571428571428),
 'Nygård_12450882': (0.19999999999999998, 1.1428571428571428),
 'Skansemyren_3139368': (0.15714285714285717, 1.1428571428571428),
 'Kalfaret_9404499': (0.19999999999999998, 1.1428571428571428),
 'Møhlenpris_9304097': (0.19999999999999998, 1.1428571428571428),
 'Starefossen_9404498': (0.19999999999999998, 1.1428571428571428),
 'Laksevåg_3148518': (0.15714285714285717, 1.1428571428571428),
 'Frydenbø_9404181': (0.15714285714285717, 1.1428571428571428),
 'Damsgård_3159313': (0.15714285714285717, 1.1428571428571428),
 'Gyldenpris_3241047'

In [604]:
bergen.cache['Bergen']

[{'adminCode1': '46',
  'lng': '5.32867',
  'distance': '0.2308',
  'geonameId': 11282303,
  'toponymName': 'Jellet Nord',
  'countryId': '3144096',
  'fcl': 'P',
  'population': 0,
  'countryCode': 'NO',
  'name': 'Jellet Nord',
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': '46'},
  'countryName': 'Norway',
  'fcodeName': 'section of populated place',
  'adminName1': 'Vestland',
  'lat': '60.39694',
  'fcode': 'PPLX'},
 {'adminCode1': '46',
  'lng': '5.32966',
  'distance': '0.24611',
  'geonameId': 11282305,
  'toponymName': 'Eidemarken',
  'countryId': '3144096',
  'fcl': 'P',
  'population': 0,
  'countryCode': 'NO',
  'name': 'Eidemarken',
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': '46'},
  'countryName': 'Norway',
  'fcodeName': 'section of populated place',
  'adminName1': 'Vestland',
  'lat': '60.39942',
  'fcode': 'PPLX'},
 {'adminCode1': '46',
  'lng': '5.32561',
  'distance': '0.31547',
  'geonameId': 9304173,
  'toponymName': 'Bergen

In [935]:
city_lat, city_lon = 56.0, -3.75
map_weather = folium.Map(location=[city_lat, city_lon], zoom_start=10)
folium.Marker([city_lat, city_lon], popup='Grangemouth').add_to(map_weather)
map_weather

In [378]:

df_e15k = pd.DataFrame(edinburgh15k.nearby_place(300, 'cities15000', 500))

Cache Hit


In [331]:
edinburgh.weather_daily(city='Grangemouth', latitude=56.0, longitude=-3.75)

WCache Hit


{'latitude': 56.0,
 'longitude': -3.75,
 'generationtime_ms': 0.25594234466552734,
 'utc_offset_seconds': 3600,
 'timezone': 'Europe/London',
 'timezone_abbreviation': 'BST',
 'elevation': 4.0,
 'daily_units': {'time': 'iso8601',
  'precipitation_sum': 'mm',
  'precipitation_hours': 'h'},
 'daily': {'time': ['2023-05-25',
   '2023-05-26',
   '2023-05-27',
   '2023-05-28',
   '2023-05-29',
   '2023-05-30',
   '2023-05-31'],
  'precipitation_sum': [0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0],
  'precipitation_hours': [0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0]}}

In [327]:
for index, row in df_e15k[['name', 'lat', 'lng']].head(10).iterrows():
#     print(row['name'], row['lat'])
    print(edinburgh.weather_daily(row['name'],row['lat'], row['lng']))



WCache Hit
{'latitude': 55.92, 'longitude': -3.3000002, 'generationtime_ms': 0.308990478515625, 'utc_offset_seconds': 3600, 'timezone': 'Europe/London', 'timezone_abbreviation': 'BST', 'elevation': 57.0, 'daily_units': {'time': 'iso8601', 'precipitation_sum': 'mm', 'precipitation_hours': 'h'}, 'daily': {'time': ['2023-05-25', '2023-05-26', '2023-05-27', '2023-05-28', '2023-05-29', '2023-05-30', '2023-05-31'], 'precipitation_sum': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'precipitation_hours': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}}
WCache Hit
{'latitude': 55.94, 'longitude': -3.48, 'generationtime_ms': 0.32198429107666016, 'utc_offset_seconds': 3600, 'timezone': 'Europe/London', 'timezone_abbreviation': 'BST', 'elevation': 69.0, 'daily_units': {'time': 'iso8601', 'precipitation_sum': 'mm', 'precipitation_hours': 'h'}, 'daily': {'time': ['2023-05-25', '2023-05-26', '2023-05-27', '2023-05-28', '2023-05-29', '2023-05-30', '2023-05-31'], 'precipitation_sum': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 

In [392]:
# checking cache for weather data
edinburgh.cache['Edinburgh']

[{'adminCode1': 'SCT',
  'lng': '-3.25609',
  'distance': '3.08977',
  'geonameId': 2652577,
  'toponymName': 'Colinton',
  'countryId': '2635167',
  'fcl': 'P',
  'population': 7637,
  'countryCode': 'GB',
  'name': 'Colinton',
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': 'SCT'},
  'countryName': 'United Kingdom',
  'fcodeName': 'populated place',
  'adminName1': 'Scotland',
  'lat': '55.90739',
  'fcode': 'PPL'},
 {'adminCode1': 'SCT',
  'lng': '-3.30845',
  'distance': '3.60446',
  'geonameId': 2651685,
  'toponymName': 'Currie',
  'countryId': '2635167',
  'fcl': 'P',
  'population': 6500,
  'countryCode': 'GB',
  'name': 'Currie',
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': 'SCT'},
  'countryName': 'United Kingdom',
  'fcodeName': 'populated place',
  'adminName1': 'Scotland',
  'lat': '55.8964',
  'fcode': 'PPL'},
 {'adminCode1': 'SCT',
  'lng': '-3.38028',
  'distance': '5.63405',
  'geonameId': 2639615,
  'toponymName': 'Ratho',
  'count

In [261]:
df_dailyupdates = pd.DataFrame(weather.json()['daily'])
df_dailyupdates

,time,precipitation_sum,precipitation_probability_max
0,2023-05-24,0.0,19
1,2023-05-25,0.0,3
2,2023-05-26,0.0,0
3,2023-05-27,0.0,3
4,2023-05-28,0.0,10
5,2023-05-29,1.2,19
6,2023-05-30,0.0,26


In [263]:
df_hourly_updates = pd.DataFrame(weather.json()['hourly'])
df_hourly_updates['cloudcover'].mean()

64.13690476190476

In [157]:
len(edinburgh100)

456

In [161]:
edin = []
for i in data['geonames']:
    if i['fcode'].startswith('PPL'):
        edin.append(i['toponymName'])
    
#     if len(data['geonames']) < max_rows:
#         break
    
#     start_row += max_rows
len(edin)

500

In [145]:
# Points of interest
radius = 1  # radius in km
lat = 55.9027  # latitude of center point
lon = -3.2492  # longitude of center point
username = ''
max_rows = 50
    # Send request to Geonames
edinburghPOI = []
url = f'http://api.geonames.org/findNearbyPOIsOSMJSON?lat={lat}&lng={lon}&radius={radius}&maxRows={max_rows}&cities=cities15000&username={username}'
response = requests.get(url)
data = response.json()
#     print(data)
# fcode for a village,town or a city is PPL
for i in data['poi']:
    if i['typeClass'] == 'place':
        edinburghPOI.append(i['name'])
    
#     if len(data['geonames']) < max_rows:
#         break
    
#     start_row += max_rows
print(edinburghPOI)

['Redford', 'Bonaly', 'Colinton']


In [45]:
# Reading the dataframe from downloaded geonames file

# with open('/Users/deepaliyadav/Downloads/GB/GB.txt', mode='r') as country:
import pandas as pd
    

In [209]:
list(data.values())[0]

[{'adminCode1': 'SCT',
  'lng': '-3.25609',
  'distance': '0.67724',
  'geonameId': 2652577,
  'toponymName': 'Colinton',
  'countryId': '2635167',
  'fcl': 'P',
  'population': 7637,
  'countryCode': 'GB',
  'name': 'Colinton',
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': 'SCT'},
  'countryName': 'United Kingdom',
  'fcodeName': 'populated place',
  'adminName1': 'Scotland',
  'lat': '55.90739',
  'fcode': 'PPL'},
 {'adminCode1': 'SCT',
  'lng': '-3.30845',
  'distance': '3.77206',
  'geonameId': 2651685,
  'toponymName': 'Currie',
  'countryId': '2635167',
  'fcl': 'P',
  'population': 6500,
  'countryCode': 'GB',
  'name': 'Currie',
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': 'SCT'},
  'countryName': 'United Kingdom',
  'fcodeName': 'populated place',
  'adminName1': 'Scotland',
  'lat': '55.8964',
  'fcode': 'PPL'},
 {'adminCode1': 'SCT',
  'lng': '-3.17814',
  'distance': '5.72575',
  'geonameId': 6930999,
  'toponymName': 'Bilston',
  'cou

In [69]:
colnames = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code', 'country code', 'cc2'
     ...: , 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date']

df = pd.read_csv('/Users/deepaliyadav/Downloads/GB/GB.txt', sep='\t', names=colnames)

/Users/deepaliyadav/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [159]:
df[df['name'] == 'Portree']

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
6651,2640006,Portree,Portree,"Port Righ,Port Rìgh,Port Rígh,Portree,Portri,b...",57.41288,-6.19418,P,PPL,GB,NaN,SCT,V3,NaN,NaN,2410,NaN,15,Europe/London,2017-06-12


In [87]:
df[((df['feature code'] == ('ADM2')) | (df['feature code'] == ('ADM3')))  & (df['admin1 code'] == 'SCT')].head(50)

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
1142,2634354,West Lothian,West Lothian,"Gorllewin Lothian,Labhaidh Thoir,Laensi-Lothia...",55.91667,-3.50000,A,ADM2,GB,NaN,SCT,W9,NaN,NaN,180130,NaN,125,Europe/London,2017-07-13
1215,2634428,Eilean Siar,Eilean Siar,"ELS,Eilean Siar,Western Isles,Western Isles Is...",57.66667,-7.16667,A,ADM2,GB,NaN,SCT,W8,NaN,NaN,26830,NaN,6,Europe/London,2022-04-18
3233,2636507,Sutherlandshire,Sutherlandshire,"Sutherland,Sutherland County",58.25000,-4.50000,A,ADM3,GB,NaN,SCT,V3,2636507,NaN,0,NaN,263,Europe/London,2008-01-16
3629,2636909,Stirling,Stirling,"County of Stirling,STG,Stirling,Stirling Count...",56.11532,-4.23386,A,ADM2,GB,NaN,SCT,W6,NaN,NaN,93750,NaN,121,Europe/London,2017-07-13
4710,2638010,Shetland Islands,Shetland Islands,"Hetland,Hit-land,Hitland,Hitlandt,Illas Shetla...",60.33333,-1.33333,A,ADM2,GB,NaN,SCT,W3,NaN,NaN,23200,NaN,134,Europe/London,2017-07-13
4926,2638228,Selkirkshire,Selkirkshire,"Selkirk,Selkirk County",55.50000,-3.00000,A,ADM3,GB,NaN,SCT,T9,2638228,NaN,0,NaN,337,Europe/London,2013-07-11
5701,2639033,Roxburghshire,Roxburghshire,"Roxburgh,Roxburgh County",55.50000,-2.50000,A,ADM3,GB,NaN,SCT,T9,2639033,NaN,0,NaN,103,Europe/London,2013-07-11
6152,2639494,Renfrewshire,Renfrewshire,"Comhairle Shiorrachd Rinn Friu,Comhairle Shior...",55.83333,-4.50000,A,ADM2,GB,NaN,SCT,W2,NaN,NaN,175930,NaN,34,Europe/London,2017-07-13
7151,2640525,Peeblesshire,Peeblesshire,"Peebles,Peebles County,Tweeddale",55.75000,-3.25000,A,ADM3,GB,NaN,SCT,T9,2640525,NaN,0,NaN,306,Europe/London,2013-07-11
7539,2640923,Orkney Islands,Orkney Islands,"Arcaibh,Illes Orcades,Illes Òrcades,Inizi Orc'...",59.00000,-3.00000,A,ADM2,GB,NaN,SCT,V9,NaN,NaN,21850,NaN,29,Europe/London,2021-09-18
